## 主修饰器
任何效果，必须且只能使用以下5个修饰器中的其中1个：
* 用多个修饰器的场合：*大概*会上面的盖掉下面的
* 不用修饰器的场合：这个效果不会被gu游执行，但是可以作为工具函数自己调用

### @ignite
标明下方函数是启动效果，可以在主要阶段使用。不需要括号（因为不需要参数）

下方函数的参数应为`c, gu`。`c`代表发动这个效果的卡，`gu`代表这场gu游（即gu游对象）。所有"操作"都由`gu`提供的接口实现

例子如下

    @ignite
    def 这张卡回到手牌(c, gu):
       gu.tohand(c)
       
    @ignite
    def 抽1张先别管为啥没有1回合1次(c, gu):
       gu.draw(c.p) # c.p就是c的控制者，draw没写第二个参数就是默认抽1张

### @move

标明下方函数是行效果，可以在战斗阶段使用，作为cost横置。例子如下

    @move
    def 行_对场上的所有怪兽造成1点伤害(c, gu):
        gu.damage(gu.mzone[0] + gu.mzone[1], 1) # gu.mzone[0]即0号玩家的怪兽区，[1]即1号玩家的怪兽区
        
    @move
    def 行然后重置的意义不明系列(c, gu):
        gu.transpose(c) # transpose在横置时重置，没有横置时横置。在函数运行时c应该已经横置了

### @common
标明下方函数是已有定义的效果（例如[贯通]之类）。下方函数的名称必须和定义里的名称完全一致（目前是瞎起的英文）。例子如下

    @common
    def through(c, gu): pass       # 贯通
    
    @common
    def block(c, gu): pass         # 阻挡
    
    @common
    def counter4(c, gu): pass      # 反击4，可以把参数写在函数的名字里（原则上不太好）
    
函数内容可以不为`pass`，那个场合会在定义运行后运行函数的内容。也可以通过`@optional`或其他修饰符来进行魔改，如

    @wangsheng
    @common
    def through(c, gu): pass       # 旺盛：贯通
    
    @optional
    @common
    def block(c, gu, chk):
        if chk: return c.T         # 横置的场合：阻挡，c.T即c是否为横置的状态

### @trigger(event, \*pattern='self')
在满足`pattern`的`event`进行后触发下方函数，下方函数的参数应在标准的`c, gu`之后再加上该`event`的参数
* `event`：函数。一般是`gu`的标准行为，如`gu.destroy`，`gu.banish`，`gu.summon`，`gu.draw`，`gu.tohand`之类。可以是自定`event`（详见自定`event`）。无论是`gu`本身的流程还是卡片效果，`event`进行后将触发下方函数


* `*pattern`：多个`'self'`或`'other'`或`None`或其他，逗号隔开。基础过滤，对照该event的参数，在**对应的位置**写上4种中的1种（或不写表示None）。默认值是`'self'`，即只触发**第一个参数**与这张卡相关的事件
 * `'self'`：该参数必须等于或包含这张卡
 * `'other'`: 该参数必须等于或包含其他卡
 * `None`：该参数可以是任何值
 * 其他：该参数必须等于这个值
 
例子如下

    @trigger(gu.summon)
    def 登场时(c, gu, summoned): xxxx
    
    @trigger(gu.leave)
    def 离场时(c, gu, leaved): xxxx
    
    @trigger(gu.attack, None, 'self') # gu.attack 的第一个参数是被攻击对象
    def 攻击时(c, gu, attacked, attacker): xxxx
    
    @trigger(gu.damage, 'self', None, None, 'battle'): # 原因是战斗
    def 受到战斗伤害时(c, gu, damaged, damager, damage, reason): xxxx

### @hook(event, \*pattern='self', priority=0)
在满足`pattern`的`event`进行前触发下方函数，一般**只在修改event时使用**，使用较困难，不正确的使用会导致程序卡死或崩溃。使用方法在`@trigger`的基础上有以下修改：

* 增加了`priority`参数（默认为0），用于处理多个`hook`的顺序。越高`priority`的`hook`会越晚执行，但如果中间有`hook`打断了这个过程，则该`hook`不会被执行。一般不用设置


* 下方函数需要返回修改过的该`event`的参数（而不是调用该`event`，这样会导致死循环；原则上不应该在`hook`里调用任何`gu`开头的函数），或返回`None`打断这个操作


* 对于需要提前判定的`event`（如`gu.summon`，需要提前判定是否能召唤，再让玩家选择是否要召唤），需要对该`event`的`chk`额外处理，详见例子

例子如下

    @hook(gu.summon):
    def 解放1只随从才能登场(c, gu, summoned, chk):
        if chk:                                     # 提前判定阶段
            if len(gu.mzone[c.p]) == 1: return None # 只有1个单位，即英雄，不能召唤
        else:                                       # 运行阶段，已经判定过了，所以不用再检查
            gu.tograve(gu.selectselfservant(c.p))   # 解放1只随从
        return (summoned, chk)                      # 返回原本的参数，供下一个hook或者召唤本身使用

    @hook(gu.draw):
    def 只要这张卡在怪兽区域存在这张卡的控制者抽卡的场合那个数量加1(c, gu, player, number): # gu.draw没有提前判定
        if player == c.p:                           # 基础过滤不能匹配控制者之类的，因为那时还没拿到`c`
            number += 1                             # c是这个函数的参数，基础过滤在这个函数上面
        return (player, number)                     # 让gu系统使用修改过的参数, **绝对不能**在这个函数里调用gu.draw

    @hook(gu.tick, 'draw'):                         # 阶段开始事件，原则上回合结束时=抽卡阶段开始时之前
    def 回合结束时(c, gu, phase):                    # gu.tick没有对应的提前判定，别忘了加phase
        xxxx
        return (phase,)                             # 忘了返回会导致回合无法结束
    
上面例子的更好写法是用`gu.tock`，专门设立的阶段结束事件（因为频繁的使用`hook`很危险）

    @trigger(gu.tock， 'battle'):
    def 战斗阶段结束也就是回合结束时(c, gu, phase):
        xxxx                                        # 不用return

## 副修饰器
这些修饰器用于模块化（简化）脚本的开发

### @loc(\*locations=gu.mzone或gu.szone)
标明下方函数生效的位置（多个的场合用逗号隔开），怪兽效果默认只在前场生效，魔法默认只在后场生效。**作为特例**，`@trigger(gu.destroy)`，即这张卡被破坏时，会自动把发动位置改为墓地（被破坏事件发生时那张卡已经在墓地了）。例子如下

    @loc(gu.grave)
    @trigger(gu.destroy, None)
    def 闪刀姬零依2效果(c, gu, destroyed, destroyer):
        if i.type&card.type.shandaoji and i.p==c.p:
            gu.summon(c)

### @onceperturn
标明下方函数为1回合1次的效果，例子如下

    @ignite
    @onceperturn
    def 一回合一次洗牌(c, gu):
        gu.shuffle(c.p)

### @attribute(\*name)
标明下方函数需要在其效果使用者上注册一个名为`name`的值（多个用逗号隔开）

在不使用`@attribute`的场合，尝试读取一个没有被初始化的值不会像lua一样会返回`nil`，而是会直接回产生`Exception`（即不处理的场合会导致程序报错跳出）。当然gu游系统本身会抓住这个`Exception`，但是这也会导致该效果不能正常运行。如果不想自己处理`Exception`或使用`hasattr`之类的，就可以用这个修饰器。

另外，卡的attribute会在区域变动时重置，这个也比较方便。例子如下

    # 回合结束时，抽这个回合这张卡被攻击的次数
    @attribute('attackedCountThisTurn')
    @trigger(gu.attack)
    def partA(c, gu):
        c.attackedCountThisTurn += 1
    
    # 两个函数中的任意一个注册就行，这个不需要@attribute
    @trigger(gu.tock, 'battle')
    def partB(c, gu):
        gu.draw(c.attackedCountThisTurn)
        c.attackedCountThisTurn = 0

### @optional
标明下方函数是需要提前判定的选发效果。确切的说，是只在特定条件下才能发动，且可以选择不发动的效果（启动效果也是选发，但不用@optional）

标明了`@optional`的函数的参数中的`c, gu`应该变成`c, gu, chk`，`chk`用于表示现在是否是判定，是判定的场合应返回是否表示这个效果能不能发动。`@optional`和`@hook`的组合比较复杂，因为此时有两个`chk`，4种可能的状态，请小心思考

例子如下

    @trigger(gu.tohand)
    @onceperturn
    def 一回合1次2张以上卡加入手卡时可以把那些卡除外(c, gu, chk, tohanded):
        if chk: return len(tohanded) >= 2
        gu.banish(tohanded)

### @wangsheng
标明下方函数是一个旺盛效果，只有在满血时才能发动。例子如下，还有很多类似的条件装饰器，不再一一举例

    @ignite
    @wangsheng
    @onceperturn
    def 旺盛1回1抽1(c, gu):
        gu.draw(c.p)

### @transpose
标明下方函数需要横置才能发动。例子如下，还有很多类似的cost装饰器，不再一一举例

    @ignite
    @transpose
    @xielv
    def 斜率横置1回1抽1(c, gu):
        gu.draw(c.p)

## 完全没有动的effect系统.jpg

In [18]:
def trigger(event, subject='self'):
    def trigger(op):
        pass
    return trigger

def hook(event, subject='self'):
    def hook(op):
        pass
    return trigger

def ignite(op):
    pass

def move(op):
    pass

def common(op):
    pass

class guProxy:
    def __getattribute__(s, name):
        pass

gu = guProxy()

## 稍微写了一点的脚本.jpg

In [19]:
@trigger(gu.summon)
def c1(c, gu, summoned):
    gu.destroy(gu.selectservant(c.p, lambda c: c.hp<=1))

@hook(gu.damage)
def c2(c, gu, damaged, damage, damager, reason):
    return (damager, damaged, damage if damaged.hp == damaged.maxhp or reason != gu.attack else damage+2)

@common
def through(c, gu):
    pass

from random import choices
@trigger(gu.destroy)
def c4(c, gu):
    t = choices(filter(gu.deck[c.p], lambda c: c.side==card.side.neutral and c.type&card.type.equip))
    gu.show(t)
    gu.tohand(gu.select(c.p, t))

@trigger(gu.destroy)
def c5(c, gu):
    gu.draw(c.p)

@trigger(gu.summon)
def c6(c, gu):
    gu.update(gu.selectservant(c.p), 0, 2, 0)

@common
def block(c, gu):
    pass

@trigger(gu.summon)
def c11(c, gu):
    gu.purify(gu.selectunit(c.p))

@common
def through(c, gu):
    pass

@trigger(gu.summon)
def c13a(c, gu):
    gu.tograve(gu.selecthand(c.p))
@trigger(gu.destroy)
def c13b(c, gu):
    gu.draw(c.p)

@trigger(gu.summon)
def c14(c, gu):
    gu.draw(c.p)

@hook(gu.damage)
def c16(c, gu, damaged, damage, damager, reason):
    gu.damage(damaged, damage-1, damager, reason)

@trigger(gu.summon)
def c17(c, gu):
    t = gu.selectunit(c.p)
    gu.diaoling(t)
    gu.update(t, 0, 0, 5)

@trigger(gu.summon)
def c18(c, gu):
    gu.damage(filter(sum(gu.mzone,[]), lambda c: c.type&card.type.servant))

@common
def block(c, gu):
    pass

@ignite
def c19(c, gu):
    gu.destroy(c)
    gu.damage(gu.selectservant(c.p), 6)

@trigger(gu.summon)
def c22a(c, gu):
    gu.damage(gu.selectunit(c.p), 1)
@trigger(gu.tock, gu.attack)
def c22b(c, gu):
    gu.damage(gu.selectunit(c.p), 1)

@trigger(gu.summon, None)
def c24a(c, gu, summoned):
    if c in summoned and len(gu.mzone[c.p]) == 2:
        gu.update(c, 2, 0, 0)
    elif c not in summoned and len(summoned) == len(gu.mzone[c.p]) - 2:
        gu.update(c, -2, 0, 0, 0)
@trigger(gu.leave, None)
def c24b(c, gu, leaved):
    if any(C.p==c.p for C in leaved) and len(gu.mzone[c.p]) == 2:
        gu.update(c, 2, 0, 0)

@common
def block(c, gu):
    pass
@ignite
def c25(c, gu):
    t = gu.selectunit(c.p)
    if not t.T:
        gu.transpose(t)

@common
def through(c, gu):
    pass

@trigger(gu.summon)
def c28(c, gu):
    gu.summon([card(gu.db[29], p=c.p), card(gu.db[29], p=c.p)])

@trigger(gu.tock, gu.attack)
def c30(c, gu):
    gu.purify(c)
    gu.recover(c, 1)

@common
def block(c, gu):
    pass
@common
def counter4(c, gu):
    pass

@trigger(gu.summon)
def e(c, gu, summoned):
    gu.destroy(gu.selectservant(c.p, lambda c: c.turn != gu.turn))
@trigger(gu.leave)
def e(c, gu, leaved):
    gu.destroy(gu.mzone[c.p][1:])

@common
def block(c, gu):
    pass

@persist
@hook(gu.use, None)
@attribute('nospellturn')
def e(c, gu, usee, usecard, chk):
    if c.nospellturn == gu.turn and chk and usecard.type&card.type.spell: return None
    return (usee, usecard, chk)
@trigger(gu.summon)
def e(c, gu, summoned):
    c.nospellturn = gu.turn
    
@trigger(gu.summon)
def e(c, gu):
    gu.purify(gu.selectunit(c.p))

@hook(gu.effect, None)
@attribute('forbidtarget')
def e(c, gu, effected):
    if effected == c.forbidtarget: return None
    return (effected,)
@trigger(gu.summon)
def e(c, gu, summoned)
    c.forbidtarget = gu.selectunit(c.p)

@ignite
@transpose
def e(c, gu):
    gu.updatemp(c.p, 2)

@trigger(gu.summon)
def e(c, gu, summoned):
    t = len(gu.grave[1-c.p])
    gu.banish(gu.grave[1-c.p])
    gu.tograve(gu.deck[1-c.p][:t])
@ignite
def e(c, gu):
    gu.tohand(c)

@trigger(gu.attack, None, 'self')
def e(c, gu, attacked, attacker):
    if not attacked.T: gu.transpose(attacked)
    gu.buffdongshang(attacked)
@move
def e(c, gu):
    gu.damage(filter(gu.mzone[1-c.p], lambda c: c.dongshang))

@trigger(gu.attack, None, 'self')
def e(c, gu):
    gu.damage(c, gu.mzone[1-c.p][0], 2)